In [1]:
import pandas as pd
from glob import glob
import json
import os

In [2]:
kzp_mappings = pd.read_parquet(
    "./data/mappings/legislation_mappings_from_kzp_scraped.parquet"
)
cpp_mappings = pd.read_parquet(
    "./data/mappings/legislation_mappings_from_cpp_scraped.parquet"
)

In [3]:
scraped_mappings = pd.concat([kzp_mappings, cpp_mappings]).reset_index(drop=True)
scraped_mappings


,title,abbr,zveza,evidencna_stevilka
0,Zakon o kazenskem postopku,ZKP,"[371. člen ZKP, 372. člen ZKP]",VS00012291
1,Kazenski zakonik,KZ-1,[235. člen KZ-1],VS00012291
2,Zakon o kazenskem postopku,ZKP,[485. člen ZKP],VS00071271
3,Zakon o kazenskem postopku,ZKP,"[269. člen ZKP, 371. člen ZKP, 395. člen ZKP]",VS00008905
4,Zakon o kazenskem postopku,ZKP,"[118. člen ZKP, 120. člen ZKP]",VS00003237
...,...,...,...,...
32069,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[2. člen 93/13/EGS, 3. člen 93/13/EGS]",VS00075210
32070,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[3. člen 93/13/EGS],VS00064778
32071,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,[4. člen 93/13/EGS],VS00046864
32072,Direktiva Sveta 93/13/EGS z dne 5. aprila 1993...,93/13/EGS,"[4. člen 93/13/EGS, 5. člen 93/13/EGS]",VS00075522


In [4]:
for evidencna_stevilka, _group in scraped_mappings.groupby("evidencna_stevilka"):
    zakonodaja_naslovi = sorted(_group["title"].values.tolist())
    zakonadaja_kratice = sorted(_group["abbr"].values.tolist())

    zveza = []
    for x in _group["zveza"].values.tolist():
        zveza.extend(x)
    zveza = sorted(zveza)

    files = glob(f"./data/scraped/*/{evidencna_stevilka}.json")
    assert len(files) == 1

    with open(files[0], "r") as f:
        data = json.load(f)

    mapping = {
        "meta": data["meta"],
        "content": data["content"],
        "processed": {
            "zakonodaja_naslovi": zakonodaja_naslovi,
            "zakonadaja_kratice": zakonadaja_kratice,
            "zveza": zveza,
            "evidencna_stevilka": evidencna_stevilka,
        },
    }

    os.makedirs("./data/training", exist_ok=True)
    with open(f"./data/training/{evidencna_stevilka}.json", "w") as f:
        json.dump(mapping, f, indent=4)